# Testing

* Given a learnt gradient, predict value of conc in samples. 
* Run in batches of size 100 so that not all 1000+ samples are loaded into memory all at once.


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
import matplotlib.patches as patches

import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
import json
import time
from io import StringIO
from datetime import datetime
import pickle
import re
import math

from sklearn.linear_model import LinearRegression
from scipy import integrate

import sys
from nmr_targeted_utils import *
from nmr_fitting_funcs import *


# Load Data

Data isn't loaded all at one shot, because that's just inviting out-of-memory error for very large datasets, but loaded (in the next section) and processed in batches of size `batch_size`.

In [5]:
# ========== define params ==========
load_dotenv()
# define paths
path_samples = os.getenv("TEST_PATH")
template_path = "./assets/lproline_ph3.csv"
bs_grad_path = "bootstrap_results_12sep2023.csv"

# proline multiplet coords
multiplets_ls = [[1.9,2.15], [2.295, 2.403], [3.25, 3.5],[4.1, 4.2]]

#signal_free_coords = [-1, 10] # signal free region is outside of these coords
normxcorr_th = 0.0 # set to this number to filter out multiplets which aren't at least normxcorr_th, i.e. poor fits
ref_pk_window = [-0.2, 0.2]
ref_pk_tolerance_window = [0,0]
search_region_padding_size = 0.02
batch_size = 100

suffix = template_path.split("/")[-1].replace(".csv", "")
fn_out_plot = f"mlgrad_{suffix}.html"
fn_out_df = f"mlgrad_{suffix}.csv"

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
folder_name = f"./results/test-{timestamp}"
print(f"Results to be stored in {folder_name}")

# ========== load data ==========
# load STD template(s)
template_df = pd.read_csv(template_path)
template_df = adjust_to_ref_peak(template_df, ref_pk_window, ref_pk_tolerance_window)

# load bootstrap gradients
d_bs_grad = pd.read_csv(bs_grad_path)
print(f"Loaded {len(d_bs_grad)} bootstrapped gradients")

matching_regions_ls = [
    [2.305, 2.306],
    [2.31, 2.316],
    [2.321, 2.3225],
    [2.331, 2.333],
    [2.342, 2.3445],
    [2.347, 2.349],
    [2.3585, 2.3605],
    [2.365, 2.3675],
    [2.3755, 2.3765],
    [2.381, 2.39]]

# load data filenames to batch up
fn_ls = []
for fn in os.listdir(path_samples):
    if ".csv" in fn:
        fn_ls.append(fn)
fn_ls = sorted(fn_ls)

batches_ls = [fn_ls[i:i+batch_size] for i in range(0, len(fn_ls), batch_size)]

counter = 1
print(f"{len(fn_ls)} samples loaded into {len(batches_ls)} batches of size:")
for batch in batches_ls:
    print(f"{counter}. {len(batch)}")
    counter += 1

# display output folder, but don't create it yet
print(f"Results to be stored in {folder_name} (not yet created)")

Results to be stored in ./results/test-20241226_1200
Loaded 5000 bootstrapped gradients
1123 samples loaded into 12 batches of size:
1. 100
2. 100
3. 100
4. 100
5. 100
6. 100
7. 100
8. 100
9. 100
10. 100
11. 100
12. 23
Results to be stored in ./results/test-20241226_1200 (not yet created)


# Run

In [7]:
# make folder
os.makedirs(folder_name, exist_ok=True)
print(f"Created folder {folder_name}")

# plot params
ticklabel_fontsize = 10

# create to store slopes and intercepts
d_lr_results_ls = []
t0 = time.perf_counter()

# get reds
red_dt = template_df.copy()
red_dt = red_dt.loc[(red_dt["ppm"]>min(multiplets_ls[1])) & (red_dt["ppm"]<max(multiplets_ls[1]))]

blue_m1_dict_ls = []
d_rho_ls = []
batch_num = 1
for batch in batches_ls:
    print(f"Processing batch {batch_num} of {len(batches_ls)}")
    # ===== load batch data =====
    df_dict = {}
    for fn in batch:
        dt = pd.read_csv(os.path.join(path_samples, fn))
        df_dict[fn.replace(".csv", "")] = adjust_to_ref_peak(
            dt, 
            ref_coords=ref_pk_window
        )

    # ===== run 1d_std_search =====
    results_dict = {}
    for k in sorted(list(df_dict.keys())):
        target_df = df_dict[k]
        results_dict[k] = do_1d_std_search(
            query_df=template_df,
            target_df=target_df,
            multiplets_ls=multiplets_ls,
            search_region_padding_size=search_region_padding_size
        )

    # get blues
    blue_m1_dict = get_blue_m1_dict(results_dict, 
                                    df_dict,
                                    mcoords=multiplets_ls[1]
                                   )
    blue_m1_dict_ls.append(blue_m1_dict)

    # ===== get corr_series_dict =====
    # get corr_series for each k, stored in corr_series_dict
    corr_series_dict = {}
    for k in sorted(list(results_dict.keys())):
        dt = get_correlation_series(red_dt, 
                                    blue_m1_dict[k].copy(),
                                    min_corr=0, 
                                    min_corr_replacement_value=0,
                                    window_size_nrows=64,
                                    exponent=8
                                   )
        corr_series_dict[k] = dt

    # ===== run LR matching =====
    df_conc = get_df_conc_lrmatching(
        results_dict=results_dict, 
        template_df=template_df.copy(), 
        df_dict=df_dict, 
        mcoords=multiplets_ls[1],
        matching_coords_ls=matching_regions_ls,
        corr_series_dict=corr_series_dict
    )
    
    d_lr_results_ls.append(df_conc)
    
    # ===== get conc_pred =====
    # get AUCs multiplied by all bootstrapped gradients in a matrix
    # get conc_pred
    auc_m = np.matmul(df_conc["auc"].values.reshape(-1, 1), 
                      d_bs_grad.values.reshape(1, -1))

    # get ave +/- 95% CI
    c = []
    for i in range(len(auc_m)):
        ave = np.average(auc_m[i])
        std = np.std(auc_m[i], ddof=1)
        ci_lower = np.percentile(auc_m[i], 2.5)
        ci_upper = np.percentile(auc_m[i], 97.5)
        c.append([df_conc["sample_name"].values[i], ave, std, ci_lower, ci_upper])

    d_concpred = pd.DataFrame(data=c, 
                              columns=["sample_name", 
                                       "conc_pred_ave", 
                                       "conc_pred_sd", 
                                       "95_ci_lower", 
                                       "95_ci_upper"]
                             )
    
    # ===== get normxcorrs =====
    c = []
    for k in sorted(list(results_dict.keys())):
        max_rho = results_dict[k]["multiplet_1"]["max_rho"][0]
        c.append([k, max_rho])
    d_max_rho = pd.DataFrame(data=c, columns=["sample_name", "normxcorr"])
    
    d_concpred = pd.merge(d_concpred, d_max_rho, on="sample_name")
    d_rho_ls.append(d_concpred[["sample_name", "normxcorr"]])
    
    # ===== plot match results =====
    fig, ax = plt.subplots(nrows=len(results_dict), # top row for LR results
                           ncols=1, 
                           figsize=(5, len(results_dict)*3)
                          )

    red_dt = template_df.copy()
    red_dt = red_dt.loc[(red_dt["ppm"]>min(multiplets_ls[1])) & (red_dt["ppm"]<max(multiplets_ls[1]))]

    i = 0
    for k in sorted(list(results_dict.keys())):
        # plot fit
        normxcorr = results_dict[k]["multiplet_1"]["max_rho"][0]
        ax[i].plot(blue_m1_dict[k].ppm.values, 
                   blue_m1_dict[k].intensity.values, c="steelblue")

        m = df_conc.loc[df_conc["sample_name"]==k]["slope"].values[0]
        c = df_conc.loc[df_conc["sample_name"]==k]["intercept"].values[0]
        ax[i].plot(red_dt.ppm.values, 
                   (red_dt.intensity.values*m)+c, 
                   c="indianred")

        ax[i].set_title(f"{i+1}. {k}\nnormxcorr={round(normxcorr, 4)}")
        
        ax[i].plot(corr_series_dict[k]["ppm"].values, 
                   corr_series_dict[k]["corr_series"].values,
                   ls="--",
                   lw=0.5,
                   c="k"
                  )

        # set bg colour
        bg_colour = "#FE9FA5" # red
        if normxcorr >= 0.85 and normxcorr < 0.9:
            bg_colour = "#FFB863" # orange
        if normxcorr >= 0.9 and normxcorr < 0.95:
            bg_colour = "#FFF263" # yellow     
        if normxcorr >= 0.95 and normxcorr < 0.99:
            bg_colour = "#96FEBF" # green
        elif normxcorr >= 0.99:
            bg_colour = "#8CDCFF" # light blue
        ax[i].set_facecolor(bg_colour)
        ax[i].set_alpha(0.7)
        plt.setp(ax[i].get_xticklabels(), fontsize=ticklabel_fontsize)
        plt.setp(ax[i].get_yticklabels(), fontsize=ticklabel_fontsize)

        # invert x-axis (ppm) to follow stupid NMR convention
        xlim_ls = list(ax[i].get_xlim())
        ax[i].set_xlim([max(xlim_ls), min(xlim_ls)])

        # draw matching regions as gray bars
        rect_height = ax[i].get_ylim()[1]
        for row in matching_regions_ls:
            # Create a rectangle patch
            rect = patches.Rectangle((min(row), 0), 
                                     max(row) - min(row), 
                                     rect_height, 
                                     edgecolor=None,
                                     facecolor='grey', 
                                     alpha=0.25)
    
            # Add the rectangle patch to the plot
            ax[i].add_patch(rect)

        i += 1

    # final bits
    plt.subplots_adjust(hspace=0.2, wspace=0)
    plt.tight_layout()
    plt.close()

    i = StringIO()
    fig.savefig(i, format="svg")
    output_svg = i.getvalue().strip().split("\n")
    output_svg = "".join(output_svg)
    svg_ls = resize_svg_bs4(output_svg, resize_coeff=0.5)
    
    # ===== prep html report =====
    # not written as a function because of the sheer number of inputs required
    html_contents = ["<html><head></head><body>"]
    html_contents.append("<ul>")
    html_contents.append(f"<li>Report generated: {datetime.today().strftime('%d %b %y, %-I:%M%p')}</li>")
    html_contents.append(f"<li>num_samples = {len(results_dict)}</li>")
    html_contents.append(f"<li>template = {template_path.split('/')[-1]}</li>")
    html_contents.append(f"<li>normxcorr threshold = {normxcorr_th}</li>")
    html_contents.append(f"<li>ref peak window = {ref_pk_window}</li>")
    html_contents.append(f"<li>ref peak tolerance window = {ref_pk_tolerance_window}</li>")
    html_contents.append(f"<li>search region padding size (ppm) = {search_region_padding_size}</li>")
    html_contents.append("</ul>")

    html_contents.append("<hr>")
    for line in output_svg:
        html_contents.append(line)
    html_contents.append("</body></html>")

    # ===== write out =====
    # write out html
    fn_out_plot = f"batch{batch_num}_viz.html"
    with open(f"./{folder_name}/{fn_out_plot}", "w") as f:
        for line in html_contents:
            f.write(line)

    # write out csvs
    dz = pd.merge(d_concpred, df_conc, on="sample_name")
    dz.to_csv(os.path.join(folder_name, f"batch{batch_num}_concpred.csv"), index=False)
    
    batch_num += 1

print("Done in %.2fs" % (time.perf_counter() - t0))

Created folder ./results/test-20241226_1200
Processing batch 1 of 12
normxcorr for AF43588 too low (0.6987968375896702), returning -1 instead
normxcorr for AF60975 too low (0.48381327436568217), returning -1 instead


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Processing batch 2 of 12


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Processing batch 3 of 12
normxcorr for AF63412_R1 too low (0.5223211498603312), returning -1 instead
normxcorr for AF63412_R2 too low (0.5219120503667406), returning -1 instead
normxcorr for AF63412_Reacquire too low (0.4747896060903554), returning -1 instead
normxcorr for AF63414 too low (0.5839849987734096), returning -1 instead
normxcorr for AF63414_Reacquire too low (0.5487009060064658), returning -1 instead


...

Processing batch 11 of 12
normxcorr for ST174 too low (0.7025507595795701), returning -1 instead


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Processing batch 12 of 12
normxcorr for ST580 too low (0.5393416053496737), returning -1 instead


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Done in 457.16s


In [ ]:
# get csv of results
dz_rho = pd.concat(d_rho_ls, axis=0).reset_index(drop=True)
dz_lr_results = pd.concat(d_lr_results_ls, axis=0).reset_index(drop=True)

# merge
dz = pd.merge(dz_rho, dz_lr_results, on="sample_name")
print(len(dz))

# write out
dz.to_csv(f"{folder_name}/conc_results.csv", index=False)